# Übung Spark mit Databricks
### Ziel der Übung
Das Ziel der Übung besteht darin, die Wikimedia-Daten aus dem Kafka Cluster zu extrahieren und auszuwerten. Es sollen folgende Fragestellungen beantwortet werden:
- Was ist das Verhältnis zwischen Bot- und Nicht-Bot Einträgen?
- Was sind die TOP-5 Domains mit den meisten Änderungen?
- Was sind die TOP-5 Accounts mit den meisten Änderungen

Zur Auswertung der Daten soll die Dataframe API verwendet werden ([API-Dokumentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html)).

In [0]:
# Für die Übung benötigte Imports
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, LongType, IntegerType
from pyspark.sql.functions import from_json, col, desc

### Lade die Wikimedia-Daten aus dem Kafka Cluster
Eine hilfreiche Dokumentation findet ihr [hier](https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html). Da die Verbindung zum Upstash-Cluster Probleme verursachen kann, geben wir euch einen Teil der Optionen vor:

```python
option("kafka.bootstrap.servers", "your_value")
option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.scram.ScramLoginModule required username=\"your_value\" password=\"your_value\";")
option("kafka.sasl.mechanism", "SCRAM-SHA-256")
option("kafka.security.protocol", "SASL_SSL")
```

In [0]:
inputDF = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "pro-horse-10077-eu2-kafka.upstash.io:9092") \
  .option("kafka.sasl.mechanism", "SCRAM-SHA-256") \
  .option("kafka.security.protocol", "SASL_SSL") \
  .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.scram.ScramLoginModule required username=\"cHJvLWhvcnNlLTEwMDc3JL_HSRE8IhJFx1QqsH5zSXl57ekHkAf4JDmbYMmSodE\" password=\"NzhiZTU1MDYtZTkxOC00NmQ4LTlmMjctNzk1ZmVjMzlmMzU5\";") \
  .option("subscribe", "wikimedia.recentchange") \
  .option("startingOffsets", "earliest") \
  .load()
  
display(inputDF)

key,value,topic,partition,offset,timestamp,timestampType
null,eyIkc2NoZW1hIjoiL21lZGlhd2lraS9yZWNlbnRjaGFuZ2UvMS4wLjAiLCJtZXRhIjp7InVyaSI6Imh0dHBzOi8vd3d3Lndpa2lkYXRhLm9yZy93aWtpL1ExMjM3NTIxMDYiLCJyZXF1ZXN0X2lkIjoiMmIzODdlNDItNTMxNS0= (truncated),wikimedia.recentchange,0,3823,2023-12-11T16:19:43.371+0000,0
null,eyIkc2NoZW1hIjoiL21lZGlhd2lraS9yZWNlbnRjaGFuZ2UvMS4wLjAiLCJtZXRhIjp7InVyaSI6Imh0dHBzOi8vd3d3Lndpa2lkYXRhLm9yZy93aWtpL1EyNjk3MzkxMyIsInJlcXVlc3RfaWQiOiIwNTViZjkxYi1mYjUyLTQ= (truncated),wikimedia.recentchange,0,3824,2023-12-11T16:19:43.372+0000,0
null,eyIkc2NoZW1hIjoiL21lZGlhd2lraS9yZWNlbnRjaGFuZ2UvMS4wLjAiLCJtZXRhIjp7InVyaSI6Imh0dHBzOi8vd3d3Lndpa2lkYXRhLm9yZy93aWtpL1E2NzYzMzg4MiIsInJlcXVlc3RfaWQiOiIzNzJjNTM3Yy05MmJiLTQ= (truncated),wikimedia.recentchange,0,3825,2023-12-11T16:19:43.374+0000,0
null,eyIkc2NoZW1hIjoiL21lZGlhd2lraS9yZWNlbnRjaGFuZ2UvMS4wLjAiLCJtZXRhIjp7InVyaSI6Imh0dHBzOi8vd3d3Lndpa2lkYXRhLm9yZy93aWtpL1E4MDg4NTU5NCIsInJlcXVlc3RfaWQiOiI4ZmRhYzZmYS1kN2JmLTQ= (truncated),wikimedia.recentchange,0,3826,2023-12-11T16:19:43.375+0000,0
null,eyIkc2NoZW1hIjoiL21lZGlhd2lraS9yZWNlbnRjaGFuZ2UvMS4wLjAiLCJtZXRhIjp7InVyaSI6Imh0dHBzOi8vamEud2lrdGlvbmFyeS5vcmcvd2lraS8lRTMlODMlODYlRTMlODMlQjMlRTMlODMlOTclRTMlODMlQUMlRTM= (truncated),wikimedia.recentchange,0,3827,2023-12-11T16:19:43.376+0000,0
null,eyIkc2NoZW1hIjoiL21lZGlhd2lraS9yZWNlbnRjaGFuZ2UvMS4wLjAiLCJtZXRhIjp7InVyaSI6Imh0dHBzOi8vd3d3Lndpa2lkYXRhLm9yZy93aWtpL1EyNzE2NDI1IiwicmVxdWVzdF9pZCI6IjU2ZjQxY2VjLTRmNDAtNDU= (truncated),wikimedia.recentchange,0,3828,2023-12-11T16:19:43.379+0000,0
null,eyIkc2NoZW1hIjoiL21lZGlhd2lraS9yZWNlbnRjaGFuZ2UvMS4wLjAiLCJtZXRhIjp7InVyaSI6Imh0dHBzOi8vd3d3Lndpa2lkYXRhLm9yZy93aWtpL1EyODk5MDg4MCIsInJlcXVlc3RfaWQiOiI3MTlhM2Y3Ny0zMGJhLTQ= (truncated),wikimedia.recentchange,0,3829,2023-12-11T16:19:43.380+0000,0
null,eyIkc2NoZW1hIjoiL21lZGlhd2lraS9yZWNlbnRjaGFuZ2UvMS4wLjAiLCJtZXRhIjp7InVyaSI6Imh0dHBzOi8vZWwud2lraXBlZGlhLm9yZy93aWtpLyVDRSVBNyVDRiU4MSVDRSVBRSVDRiU4MyVDRiU4NCVDRSVCNyVDRiU= (truncated),wikimedia.recentchange,0,3830,2023-12-11T16:19:43.382+0000,0
null,eyIkc2NoZW1hIjoiL21lZGlhd2lraS9yZWNlbnRjaGFuZ2UvMS4wLjAiLCJtZXRhIjp7InVyaSI6Imh0dHBzOi8vY29tbW9ucy53aWtpbWVkaWEub3JnL3dpa2kvQ2F0ZWdvcnk6VmVoaWNsZXNfaW5fVmlsbml1cyIsInJlcXU= (truncated),wikimedia.recentchange,0,3831,2023-12-11T16:19:43.383+0000,0
null,eyIkc2NoZW1hIjoiL21lZGlhd2lraS9yZWNlbnRjaGFuZ2UvMS4wLjAiLCJtZXRhIjp7InVyaSI6Imh0dHBzOi8vZW4ud2lrdGlvbmFyeS5vcmcvd2lraS90b2Fzc2VtIiwicmVxdWVzdF9pZCI6ImVmMjUxNjQyLTI1YzAtNDk= (truncated),wikimedia.recentchange,0,3832,2023-12-11T16:19:43.384+0000,0


### Umwandlung der Daten in das Ursprungsformat
Bei diesem Abschnitt gibt es für euch nichts zu tun. Kafka übermittelt die Daten in binärer Form, weshalb wir sie hier in das ursprüngliche JSON-Format umwandeln.

In [0]:
schema = StructType([
  StructField("schema", StringType()),
  StructField("meta", StructType([
    StructField("uri", StringType()),
    StructField("request_id", StringType()),
    StructField("id", StringType()),
    StructField("dt", StringType()),
    StructField("domain", StringType()),
    StructField("stream", StringType()),
    StructField("topic", StringType()),
    StructField("partition", IntegerType()),
    StructField("offset", LongType())
  ])),
  StructField("id", LongType()),
  StructField("type", StringType()),
  StructField("namespace", IntegerType()),
  StructField("title", StringType()),
  StructField("comment", StringType()),
  StructField("timestamp", LongType()),
  StructField("user", StringType()),
  StructField("bot", BooleanType()),
  StructField("minor", BooleanType()),
  StructField("patrolled", BooleanType()),
  StructField("length", StructType([
    StructField("old", IntegerType()),
    StructField("new", IntegerType())
  ])),
  StructField("revision", StructType([
    StructField("old", LongType()),
    StructField("new", LongType())
  ])),
  StructField("server_url", StringType()),
  StructField("server_name", StringType()),
  StructField("server_script_path", StringType()),
  StructField("wiki", StringType()),
  StructField("parsedcomment", StringType())
])

parsedValueDF = inputDF \
  .select(from_json(col("value").cast("string"), schema).alias("parsed_value"))

display(parsedValueDF)

parsed_value
"List(null, List(https://www.wikidata.org/wiki/Q123752106, 2b387e42-5315-463d-9977-99118aba208e, 940fafc3-018b-4d41-9719-0503a6debc75, 2023-12-11T16:19:43Z, www.wikidata.org, mediawiki.recentchange, codfw.mediawiki.recentchange, 0, 834622103), 2089991279, edit, 0, Q123752106, /* wbsetlabel-add:1|pt */ La Caseta'l Rublín, #quickstatements; #temporary_batch_1702311436730, 1702311583, YoaR, false, false, true, List(5110, 5188), List(2028026368, 2028026388), https://www.wikidata.org, www.wikidata.org, /w, wikidatawiki, ‎Amestada la etiqueta en [pt]: La Caseta'l Rublín, #quickstatements; #temporary_batch_1702311436730)"
"List(null, List(https://www.wikidata.org/wiki/Q26973913, 055bf91b-fb52-4761-ac2e-10c676368729, 100b35af-71dc-4a2a-82ec-d733f3c400e6, 2023-12-11T16:19:43Z, www.wikidata.org, mediawiki.recentchange, codfw.mediawiki.recentchange, 0, 834622104), 2089991282, edit, 0, Q26973913, /* wbsetdescription-add:1|af */ pluimbalspeler, [[:toollabs:quickstatements/#/batch/218668|batch #218668]], 1702311583, Florentyna, false, false, true, List(29958, 30031), List(2027880024, 2028026392), https://www.wikidata.org, www.wikidata.org, /w, wikidatawiki, ‎Added [af] description: pluimbalspeler, batch #218668)"
"List(null, List(https://www.wikidata.org/wiki/Q67633882, 372c537c-92bb-4052-81d7-96d9ba3c5509, fc1d30e6-c269-4fb2-b859-e096d944168d, 2023-12-11T16:19:43Z, www.wikidata.org, mediawiki.recentchange, codfw.mediawiki.recentchange, 0, 834622105), 2089991281, edit, 0, Q67633882, /* wbsetdescription-add:1|ryu */ 天川原ぬすりー, [[:toollabs:quickstatements/#/batch/218645|batch #218645]], 1702311583, Prefuture, false, false, true, List(7215, 7297), List(1719405584, 2028026391), https://www.wikidata.org, www.wikidata.org, /w, wikidatawiki, ‎[ryu]の説明を追加: 天川原ぬすりー, batch #218645)"
"List(null, List(https://www.wikidata.org/wiki/Q80885594, 8fdac6fa-d7bf-42aa-8c2c-835eb75fb28f, f224991d-d7ee-4c3a-90e2-6b83b779ae77, 2023-12-11T16:19:43Z, www.wikidata.org, mediawiki.recentchange, codfw.mediawiki.recentchange, 0, 834622106), 2089991280, edit, 0, Q80885594, /* wbsetdescription-add:1|ryu */ HII領域, [[:toollabs:quickstatements/#/batch/218644|batch #218644]], 1702311583, Prefuture, false, false, true, List(9399, 9468), List(2017573174, 2028026390), https://www.wikidata.org, www.wikidata.org, /w, wikidatawiki, ‎[ryu]の説明を追加: HII領域, batch #218644)"
"List(null, List(https://ja.wiktionary.org/wiki/%E3%83%86%E3%83%B3%E3%83%97%E3%83%AC%E3%83%BC%E3%83%88:ojp-pron-noun-2-1, 6cd1ed34-4cd1-4819-8374-de9e46ac1d81, 5bd2b2da-68d6-4886-a7ad-89dff3ac9c43, 2023-12-11T16:19:43Z, ja.wiktionary.org, mediawiki.recentchange, codfw.mediawiki.recentchange, 0, 834622107), 3145557, edit, 10, テンプレート:ojp-pron-noun-2-1, , 1702311583, M-30722, false, true, true, List(7677, 7677), List(1627698, 1852645), https://ja.wiktionary.org, ja.wiktionary.org, /w, jawiktionary, )"
"List(null, List(https://www.wikidata.org/wiki/Q2716425, 56f41cec-4f40-4571-aed2-5849aa130e0d, 38ecdf4a-bdc2-4fd9-853a-114da7f48d30, 2023-12-11T16:19:43Z, www.wikidata.org, mediawiki.recentchange, codfw.mediawiki.recentchange, 0, 834622108), 2089991283, edit, 0, Q2716425, /* wbsetlabel-set:1|vi */ Fallugia, 1702311583, AdamSeattle, false, false, true, List(34203, 34193), List(1945684091, 2028026389), https://www.wikidata.org, www.wikidata.org, /w, wikidatawiki, ‎Changed Vietnamese label: Fallugia)"
"List(null, List(https://www.wikidata.org/wiki/Q28990880, 719a3f77-30ba-42cd-83ed-d866393c5dbf, 43353367-6f38-4ee7-8dc8-3a0194cde41b, 2023-12-11T16:19:43Z, www.wikidata.org, mediawiki.recentchange, codfw.mediawiki.recentchange, 0, 834622109), 2089991284, edit, 0, Q28990880, /* wbsetdescription-add:1|uk */ бактеріальний ген, наявний у Thermanaerovibrio acidaminovorans DSM 6589, [[:toollabs:quickstatements/#/batch/218694|batch #218694]], 1702311583, Renamerr, false, false, true, List(15049, 15203), List(949843826, 2028026393), https://www.wikidata.org, www.wikidata.org, /w, wikidatawiki, ‎Додано опис [uk]: бактеріальний 

### Was ist das Verhältnis zwischen Bot und Nicht-Bot Einträgen?

Ermittelt das Verhältnis zwischen Bot und Nicht-Bot Einträgen. Stellt das Ergebnis anschließend in einem Pie-Chart dar. <br> Die relevanten Spark-Befehle findet ihr [hier](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.groupby.GroupBy.count.html).


In [0]:
botCountDF = parsedValueDF \
    .groupBy(col("parsed_value.bot")) \
    .count()

display(botCountDF)

Databricks visualization. Run in Databricks to view.

bot,count
true,869
false,1661


### Was sind die TOP-5 Domains mit den meisten Änderungen?
Ermittelt die TOP-5 Domains mit den meisten Änderungen. Stellt das Ergebnis anschließend in einem Bar-Chart dar.

In [0]:
changesPerDomainDF = parsedValueDF \
    .groupBy(col("parsed_value.meta.domain")) \
    .count() \
    .orderBy(desc("count")) \
    .limit(5)

display(changesPerDomainDF)

Databricks visualization. Run in Databricks to view.

domain,count
commons.wikimedia.org,828
www.wikidata.org,592
en.wikipedia.org,298
en.wiktionary.org,221
id.wikipedia.org,90


### Was sind die TOP-5 Accounts mit den meisten Änderungen?
Ermittelt die TOP-5 Accounts mit den meisten Änderungen. Stellt das Ergebnis anschließend in einem Bar-Chart dar.

In [0]:
changesPerAccountDF = parsedValueDF \
    .groupBy(col("parsed_value.user")) \
    .count() \
    .orderBy(desc("count")) \
    .limit(5)
    
display(changesPerAccountDF)

Databricks visualization. Run in Databricks to view.

user,count
WingerBot,213
PMG,157
GeographBot,137
Florentyna,90
Prefuture,88
